In [2]:
import pandas as pd
df = pd.read_csv("D:\G95-Dataset\phase2cleaneddataset.csv",encoding='cp1252')
start_Stime = 1421927443  
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
overallG_5_5_5_4_5_2 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    overallG_5_5_5_4_5_2.add_node(src_node)
    overallG_5_5_5_4_5_2.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    overallG_5_5_5_4_5_2.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonoverallG_5_5_5_4_5_2 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonoverallG_5_5_5_4_5_2.add_node(src_node)
    nonoverallG_5_5_5_4_5_2.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_5_5_5_4_5_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_5_5_5_4_5_1.add_node(src_node)
    overallG_5_5_5_4_5_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_5_5_5_4_5_1.add_edge(src_node, dst_node, weight=edge_weight)

C:\Users\HP\AppData\Local\Temp\ipykernel_12548\3393257167.py:2: DtypeWarning: Columns (1,3,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:\G95-Dataset\phase2cleaneddataset.csv",encoding='cp1252')


In [3]:
import networkx as nx

# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(overallG_5_5_5_4_5_2)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonoverallG_5_5_5_4_5_2)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_5_5_5_4_5_1)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(overallG_5_5_5_4_5_2)
avg_clustering_non_attack = nx.average_clustering(nonoverallG_5_5_5_4_5_2)
avg_clustering_overall = nx.average_clustering(overallG_5_5_5_4_5_1)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(overallG_5_5_5_4_5_2.degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_degree_overall = sum(dict(overallG_5_5_5_4_5_1.degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(overallG_5_5_5_4_5_2.in_degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_in_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.in_degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_in_degree_overall = sum(dict(overallG_5_5_5_4_5_1.in_degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(overallG_5_5_5_4_5_2.out_degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_out_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.out_degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_out_degree_overall = sum(dict(overallG_5_5_5_4_5_1.out_degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_pagerank_non_attack = sum(nx.pagerank(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_pagerank_overall = sum(nx.pagerank(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(overallG_5_5_5_4_5_2)
avg_shortest_path_non_attack = calculate_diameter(nonoverallG_5_5_5_4_5_2)
avg_shortest_path_overall = calculate_diameter(overallG_5_5_5_4_5_1)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(overallG_5_5_5_4_5_2)
density_non_attack = nx.density(nonoverallG_5_5_5_4_5_2)
density_overall = nx.density(overallG_5_5_5_4_5_1)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(overallG_5_5_5_4_5_2)
diameter_non_attack = calculate_diameter(nonoverallG_5_5_5_4_5_2)
diameter_overall = calculate_diameter(overallG_5_5_5_4_5_1)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(overallG_5_5_5_4_5_2)
num_louvain_non_attack = detect_communities(nonoverallG_5_5_5_4_5_2)
num_louvain_overall = detect_communities(overallG_5_5_5_4_5_1)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(overallG_5_5_5_4_5_2)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonoverallG_5_5_5_4_5_2)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_5_5_5_4_5_1)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(overallG_5_5_5_4_5_2)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonoverallG_5_5_5_4_5_2)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_5_5_5_4_5_1)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(overallG_5_5_5_4_5_2)
reciprocity_non_attack = calculate_reciprocity(nonoverallG_5_5_5_4_5_2)
reciprocity_overall = calculate_reciprocity(overallG_5_5_5_4_5_1)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.06711125801849091
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.08709671614965521
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 9.053978033037136e-11
Average closeness centrality for attack graph: 0.012818686796752583
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0006729976242281746
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.0252100840336134
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.997307001795332
Average degree centrality for attack graph: 0.025637373593505174
Average degree centrality for non-attack graph: 0.0
Average degree ce

In [9]:
start_Stime = 1424219889  
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
overallG_5_5_5_4_5_2 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    overallG_5_5_5_4_5_2.add_node(src_node)
    overallG_5_5_5_4_5_2.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    overallG_5_5_5_4_5_2.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonoverallG_5_5_5_4_5_2 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonoverallG_5_5_5_4_5_2.add_node(src_node)
    nonoverallG_5_5_5_4_5_2.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_5_5_5_4_5_1 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_5_5_5_4_5_1.add_node(src_node)
    overallG_5_5_5_4_5_1.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_5_5_5_4_5_1.add_edge(src_node, dst_node, weight=edge_weight)

In [10]:
import networkx as nx

# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(overallG_5_5_5_4_5_2)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonoverallG_5_5_5_4_5_2)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_5_5_5_4_5_1)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(overallG_5_5_5_4_5_2)
avg_clustering_non_attack = nx.average_clustering(nonoverallG_5_5_5_4_5_2)
avg_clustering_overall = nx.average_clustering(overallG_5_5_5_4_5_1)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(overallG_5_5_5_4_5_2.degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_degree_overall = sum(dict(overallG_5_5_5_4_5_1.degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(overallG_5_5_5_4_5_2.in_degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_in_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.in_degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_in_degree_overall = sum(dict(overallG_5_5_5_4_5_1.in_degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(overallG_5_5_5_4_5_2.out_degree()).values()) / len(overallG_5_5_5_4_5_2)
avg_out_degree_non_attack = sum(dict(nonoverallG_5_5_5_4_5_2.out_degree()).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_out_degree_overall = sum(dict(overallG_5_5_5_4_5_1.out_degree()).values()) / len(overallG_5_5_5_4_5_1)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(overallG_5_5_5_4_5_2).values()) / len(overallG_5_5_5_4_5_2)
avg_pagerank_non_attack = sum(nx.pagerank(nonoverallG_5_5_5_4_5_2).values()) / len(nonoverallG_5_5_5_4_5_2)
avg_pagerank_overall = sum(nx.pagerank(overallG_5_5_5_4_5_1).values()) / len(overallG_5_5_5_4_5_1)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(overallG_5_5_5_4_5_2)
avg_shortest_path_non_attack = calculate_diameter(nonoverallG_5_5_5_4_5_2)
avg_shortest_path_overall = calculate_diameter(overallG_5_5_5_4_5_1)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(overallG_5_5_5_4_5_2)
density_non_attack = nx.density(nonoverallG_5_5_5_4_5_2)
density_overall = nx.density(overallG_5_5_5_4_5_1)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(overallG_5_5_5_4_5_2)
diameter_non_attack = calculate_diameter(nonoverallG_5_5_5_4_5_2)
diameter_overall = calculate_diameter(overallG_5_5_5_4_5_1)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(overallG_5_5_5_4_5_2)
num_louvain_non_attack = detect_communities(nonoverallG_5_5_5_4_5_2)
num_louvain_overall = detect_communities(overallG_5_5_5_4_5_1)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(overallG_5_5_5_4_5_2)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonoverallG_5_5_5_4_5_2)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_5_5_5_4_5_1)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(overallG_5_5_5_4_5_2)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonoverallG_5_5_5_4_5_2)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_5_5_5_4_5_1)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(overallG_5_5_5_4_5_2)
reciprocity_non_attack = calculate_reciprocity(nonoverallG_5_5_5_4_5_2)
reciprocity_overall = calculate_reciprocity(overallG_5_5_5_4_5_1)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.16237189614425643
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.06345522778543293
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 2.672754445795599e-10
Average closeness centrality for attack graph: 0.015012510425354437
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0006506013526446518
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.272727272727273
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.9125
Average degree centrality for attack graph: 0.03002502085070897
Average degree centrality for non-attack graph: 0.0
Average degree centrality for 

In [17]:
start_Stime = 1421927743    
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
AttackG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    AttackG_5_5_5_4.add_node(src_node)
    AttackG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    AttackG_5_5_5_4.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonAttackG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonAttackG_5_5_5_5_5_4.add_node(src_node)
    nonAttackG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_5_5_5_4.add_node(src_node)
    overallG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_5_5_5_4.add_edge(src_node, dst_node, weight=edge_weight)

In [18]:
import networkx as nx

# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(AttackG_5_5_5_4)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonAttackG_5_5_5_4)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_5_5_5_4)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(AttackG_5_5_5_4)
avg_clustering_non_attack = nx.average_clustering(nonAttackG_5_5_5_4)
avg_clustering_overall = nx.average_clustering(overallG_5_5_5_4)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(AttackG_5_5_5_4.degree()).values()) / len(AttackG_5_5_5_4)
avg_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.degree()).values()) / len(nonAttackG_5_5_5_4)
avg_degree_overall = sum(dict(overallG_5_5_5_4.degree()).values()) / len(overallG_5_5_5_4)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(AttackG_5_5_5_4.in_degree()).values()) / len(AttackG_5_5_5_4)
avg_in_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.in_degree()).values()) / len(nonAttackG_5_5_5_4)
avg_in_degree_overall = sum(dict(overallG_5_5_5_4.in_degree()).values()) / len(overallG_5_5_5_4)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(AttackG_5_5_5_4.out_degree()).values()) / len(AttackG_5_5_5_4)
avg_out_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.out_degree()).values()) / len(nonAttackG_5_5_5_4)
avg_out_degree_overall = sum(dict(overallG_5_5_5_4.out_degree()).values()) / len(overallG_5_5_5_4)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_pagerank_non_attack = sum(nx.pagerank(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_pagerank_overall = sum(nx.pagerank(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(AttackG_5_5_5_4)
avg_shortest_path_non_attack = calculate_diameter(nonAttackG_5_5_5_4)
avg_shortest_path_overall = calculate_diameter(overallG_5_5_5_4)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(AttackG_5_5_5_4)
density_non_attack = nx.density(nonAttackG_5_5_5_4)
density_overall = nx.density(overallG_5_5_5_4)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(AttackG_5_5_5_4)
diameter_non_attack = calculate_diameter(nonAttackG_5_5_5_4)
diameter_overall = calculate_diameter(overallG_5_5_5_4)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(AttackG_5_5_5_4)
num_louvain_non_attack = detect_communities(nonAttackG_5_5_5_4)
num_louvain_overall = detect_communities(overallG_5_5_5_4)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(AttackG_5_5_5_4)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonAttackG_5_5_5_4)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_5_5_5_4)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(AttackG_5_5_5_4)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonAttackG_5_5_5_4)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_5_5_5_4)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(AttackG_5_5_5_4)
reciprocity_non_attack = calculate_reciprocity(nonAttackG_5_5_5_4)
reciprocity_overall = calculate_reciprocity(overallG_5_5_5_4)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.06026930141104398
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.10036701534225137
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 2.511284464274069e-10
Average closeness centrality for attack graph: 0.013388406209941587
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0007708217314293511
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.1596638655462184
Average degree for non-attack graph: 0.0
Average degree for overall graph: 3.078078078078078
Average degree centrality for attack graph: 0.026776812419883184
Average degree centrality for non-attack graph: 0.0
Average degree ce

In [7]:
start_Stime = 1424237559    
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
AttackG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    AttackG_5_5_5_4.add_node(src_node)
    AttackG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    AttackG_5_5_5_4.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonAttackG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonAttackG_5_5_5_4.add_node(src_node)
    nonAttackG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_5_5_5_4 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_5_5_5_4.add_node(src_node)
    overallG_5_5_5_4.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_5_5_5_4.add_edge(src_node, dst_node, weight=edge_weight)

In [8]:
import networkx as nx

# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(AttackG_5_5_5_4)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonAttackG_5_5_5_4)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_5_5_5_4)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(AttackG_5_5_5_4)
avg_clustering_non_attack = nx.average_clustering(nonAttackG_5_5_5_4)
avg_clustering_overall = nx.average_clustering(overallG_5_5_5_4)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(AttackG_5_5_5_4.degree()).values()) / len(AttackG_5_5_5_4)
avg_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.degree()).values()) / len(nonAttackG_5_5_5_4)
avg_degree_overall = sum(dict(overallG_5_5_5_4.degree()).values()) / len(overallG_5_5_5_4)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(AttackG_5_5_5_4.in_degree()).values()) / len(AttackG_5_5_5_4)
avg_in_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.in_degree()).values()) / len(nonAttackG_5_5_5_4)
avg_in_degree_overall = sum(dict(overallG_5_5_5_4.in_degree()).values()) / len(overallG_5_5_5_4)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(AttackG_5_5_5_4.out_degree()).values()) / len(AttackG_5_5_5_4)
avg_out_degree_non_attack = sum(dict(nonAttackG_5_5_5_4.out_degree()).values()) / len(nonAttackG_5_5_5_4)
avg_out_degree_overall = sum(dict(overallG_5_5_5_4.out_degree()).values()) / len(overallG_5_5_5_4)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(AttackG_5_5_5_4).values()) / len(AttackG_5_5_5_4)
avg_pagerank_non_attack = sum(nx.pagerank(nonAttackG_5_5_5_4).values()) / len(nonAttackG_5_5_5_4)
avg_pagerank_overall = sum(nx.pagerank(overallG_5_5_5_4).values()) / len(overallG_5_5_5_4)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(AttackG_5_5_5_4)
avg_shortest_path_non_attack = calculate_diameter(nonAttackG_5_5_5_4)
avg_shortest_path_overall = calculate_diameter(overallG_5_5_5_4)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(AttackG_5_5_5_4)
density_non_attack = nx.density(nonAttackG_5_5_5_4)
density_overall = nx.density(overallG_5_5_5_4)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(AttackG_5_5_5_4)
diameter_non_attack = calculate_diameter(nonAttackG_5_5_5_4)
diameter_overall = calculate_diameter(overallG_5_5_5_4)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(AttackG_5_5_5_4)
num_louvain_non_attack = detect_communities(nonAttackG_5_5_5_4)
num_louvain_overall = detect_communities(overallG_5_5_5_4)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(AttackG_5_5_5_4)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonAttackG_5_5_5_4)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_5_5_5_4)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(AttackG_5_5_5_4)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonAttackG_5_5_5_4)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_5_5_5_4)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(c)
reciprocity_non_attack = calculate_reciprocity(nonAttackG_5_5_5_4)
reciprocity_overall = calculate_reciprocity(overallG_5_5_5_4)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.07550129301086408
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.04551724651672302
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 1.6795949823315928e-10
Average closeness centrality for attack graph: 0.013704686118479198
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0005175836538363005
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.1794871794871793
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.7051625239005737
Average degree centrality for attack graph: 0.027409372236958513
Average degree centrality for non-attack graph: 0.0
Average degree 

In [9]:
start_Stime = 1424237859      
df1 = df[(df['Stime'] >= start_Stime) & (df['Stime'] <= (start_Stime + 300))]

df1_label_true = df1[df1['Label'] == 1]
df1_label_false = df1[df1['Label'] == 0]
df1_label_false_grouped = df1_label_false.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_label_true_grouped = df1_label_true.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')
df1_grouped = df1.groupby(['srcip', 'sport', 'dstip', 'dsport']).size().reset_index(name='count')

import networkx as nx
# Create an empty graph
AttackG_5_5 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    AttackG_5_5.add_node(src_node)
    AttackG_5_5.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_true_grouped)):
    src_node = str(df1_label_true_grouped['srcip'][i]) + ':' + str(df1_label_true_grouped['sport'][i])
    dst_node = str(df1_label_true_grouped['dstip'][i]) + ':' + str(df1_label_true_grouped['dsport'][i])
    edge_weight = df1_label_true_grouped['count'][i]
    AttackG_5_5.add_edge(src_node, dst_node, weight=edge_weight)


import networkx as nx
# Create an empty graph
nonAttackG_5_5 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    nonAttackG_5_5.add_node(src_node)
    nonAttackG_5_5.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_label_false_grouped)):
    src_node = str(df1_label_false_grouped['srcip'][i]) + ':' + str(df1_label_false_grouped['sport'][i])
    dst_node = str(df1_label_false_grouped['dstip'][i]) + ':' + str(df1_label_false_grouped['dsport'][i])
    edge_weight = df1_label_false_grouped['count'][i]

import networkx as nx
# Create an empty graph
overallG_5_5 = nx.DiGraph()

# Add nodes to the graph
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    overallG_5_5.add_node(src_node)
    overallG_5_5.add_node(dst_node)

# Add edges to the graph with edge weight as count
for i in range(len(df1_grouped)):
    src_node = str(df1_grouped['srcip'][i]) + ':' + str(df1_grouped['sport'][i])
    dst_node = str(df1_grouped['dstip'][i]) + ':' + str(df1_grouped['dsport'][i])
    edge_weight = df1_grouped['count'][i]
    overallG_5_5.add_edge(src_node, dst_node, weight=edge_weight)

In [10]:
# Function to calculate diameter for a graph
def calculate_diameter(G):
    try:
        if nx.is_directed(G):
            # If the graph is directed, we find the largest weakly connected component
            largest_component = max(nx.weakly_connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        else:
            # For undirected graphs, we find the largest connected component
            largest_component = max(nx.connected_components(G), key=len)
            largest_component_subgraph = G.subgraph(largest_component)
        
        # Calculate the diameter of the largest component
        diameter = nx.diameter(largest_component_subgraph)
    except nx.NetworkXError:
        diameter = None
    return diameter

# Function to calculate reciprocity with error handling
def calculate_reciprocity(G):
    try:
        # Check if the graph is empty
        if len(G) == 0:
            return None  # Return None if the graph is empty
        
        # Calculate reciprocity
        reciprocity = nx.reciprocity(G)
    except nx.NetworkXError:
        reciprocity = None  
    return reciprocity

# Function to detect communities using the greedy modularity algorithm
def detect_communities(G):
    try:
        # Check if the graph has edges
        if G.size() == 0:
            return 0  # Return 0 communities if the graph is empty
        
        # Detect communities using the greedy modularity algorithm
        communities = list(nx.algorithms.community.greedy_modularity_communities(G))
        num_communities = len(communities)
    except ZeroDivisionError:
        num_communities = 0  # Return 0 communities if a division by zero error occurs
    return num_communities





# 1. Assortativity coefficient
assortativity_attack = nx.degree_assortativity_coefficient(AttackG_5_5)
assortativity_non_attack = nx.degree_assortativity_coefficient(nonAttackG_5_5)
assortativity_overall = nx.degree_assortativity_coefficient(overallG_5_5)

print("Assortativity coefficient for attack graph:", assortativity_attack)
print("Assortativity coefficient for non-attack graph:", assortativity_non_attack)
print("Assortativity coefficient for overall graph:", assortativity_overall)

# 2. Average betweenness centrality
avg_betweenness_attack = sum(nx.betweenness_centrality(AttackG_5_5).values()) / len(AttackG_5_5)
avg_betweenness_non_attack = sum(nx.betweenness_centrality(nonAttackG_5_5).values()) / len(nonAttackG_5_5)
avg_betweenness_overall = sum(nx.betweenness_centrality(overallG_5_5).values()) / len(overallG_5_5)

print("Average betweenness centrality for attack graph:", avg_betweenness_attack)
print("Average betweenness centrality for non-attack graph:", avg_betweenness_non_attack)
print("Average betweenness centrality for overall graph:", avg_betweenness_overall)

# 3. Average closeness centrality
avg_closeness_attack = sum(nx.closeness_centrality(AttackG_5_5).values()) / len(AttackG_5_5)
avg_closeness_non_attack = sum(nx.closeness_centrality(nonAttackG_5_5).values()) / len(nonAttackG_5_5)
avg_closeness_overall = sum(nx.closeness_centrality(overallG_5_5).values()) / len(overallG_5_5)

print("Average closeness centrality for attack graph:", avg_closeness_attack)
print("Average closeness centrality for non-attack graph:", avg_closeness_non_attack)
print("Average closeness centrality for overall graph:", avg_closeness_overall)

# 4. Average clustering coefficient
avg_clustering_attack = nx.average_clustering(AttackG_5_5)
avg_clustering_non_attack = nx.average_clustering(nonAttackG_5_5)
avg_clustering_overall = nx.average_clustering(overallG_5_5)

print("Average clustering coefficient for attack graph:", avg_clustering_attack)
print("Average clustering coefficient for non-attack graph:", avg_clustering_non_attack)
print("Average clustering coefficient for overall graph:", avg_clustering_overall)

# 5. Average degree
avg_degree_attack = sum(dict(AttackG_5_5.degree()).values()) / len(AttackG_5_5)
avg_degree_non_attack = sum(dict(nonAttackG_5_5.degree()).values()) / len(nonAttackG_5_5)
avg_degree_overall = sum(dict(overallG_5_5.degree()).values()) / len(overallG_5_5)

print("Average degree for attack graph:", avg_degree_attack)
print("Average degree for non-attack graph:", avg_degree_non_attack)
print("Average degree for overall graph:", avg_degree_overall)

# 6. Average degree centrality
avg_degree_centrality_attack = sum(nx.degree_centrality(AttackG_5_5).values()) / len(AttackG_5_5)
avg_degree_centrality_non_attack = sum(nx.degree_centrality(nonAttackG_5_5).values()) / len(nonAttackG_5_5)
avg_degree_centrality_overall = sum(nx.degree_centrality(overallG_5_5).values()) / len(overallG_5_5)

print("Average degree centrality for attack graph:", avg_degree_centrality_attack)
print("Average degree centrality for non-attack graph:", avg_degree_centrality_non_attack)
print("Average degree centrality for overall graph:", avg_degree_centrality_overall)

# 7. Average eigenvector centrality
avg_eigenvector_attack = sum(nx.eigenvector_centrality_numpy(AttackG_5_5).values()) / len(AttackG_5_5)
avg_eigenvector_non_attack = sum(nx.eigenvector_centrality_numpy(nonAttackG_5_5).values()) / len(nonAttackG_5_5)
avg_eigenvector_overall = sum(nx.eigenvector_centrality_numpy(overallG_5_5).values()) / len(overallG_5_5)

print("Average eigenvector centrality for attack graph:", avg_eigenvector_attack)
print("Average eigenvector centrality for non-attack graph:", avg_eigenvector_non_attack)
print("Average eigenvector centrality for overall graph:", avg_eigenvector_overall)

# 8. Average in-degree
avg_in_degree_attack = sum(dict(AttackG_5_5.in_degree()).values()) / len(AttackG_5_5)
avg_in_degree_non_attack = sum(dict(nonAttackG_5_5.in_degree()).values()) / len(nonAttackG_5_5)
avg_in_degree_overall = sum(dict(overallG_5_5.in_degree()).values()) / len(overallG_5_5)

print("Average in-degree for attack graph:", avg_in_degree_attack)
print("Average in-degree for non-attack graph:", avg_in_degree_non_attack)
print("Average in-degree for overall graph:", avg_in_degree_overall)

# 9. Average out-degree
avg_out_degree_attack = sum(dict(AttackG_5_5.out_degree()).values()) / len(AttackG_5_5)
avg_out_degree_non_attack = sum(dict(nonAttackG_5_5.out_degree()).values()) / len(nonAttackG_5_5)
avg_out_degree_overall = sum(dict(overallG_5_5.out_degree()).values()) / len(overallG_5_5)

print("Average out-degree for attack graph:", avg_out_degree_attack)
print("Average out-degree for non-attack graph:", avg_out_degree_non_attack)
print("Average out-degree for overall graph:", avg_out_degree_overall)

# 10. Average PageRank score
avg_pagerank_attack = sum(nx.pagerank(AttackG_5_5).values()) / len(AttackG_5_5)
avg_pagerank_non_attack = sum(nx.pagerank(nonAttackG_5_5).values()) / len(nonAttackG_5_5)
avg_pagerank_overall = sum(nx.pagerank(overallG_5_5).values()) / len(overallG_5_5)

print("Average PageRank score for attack graph:", avg_pagerank_attack)
print("Average PageRank score for non-attack graph:", avg_pagerank_non_attack)
print("Average PageRank score for overall graph:", avg_pagerank_overall)



# 11. Average shortest path length
avg_shortest_path_attack = calculate_diameter(AttackG_5_5)
avg_shortest_path_non_attack = calculate_diameter(nonAttackG_5_5)
avg_shortest_path_overall = calculate_diameter(overallG_5_5)

print("Average shortest path length for attack graph:", avg_shortest_path_attack)
print("Average shortest path length for non-attack graph:", avg_shortest_path_non_attack)
print("Average shortest path length for overall graph:", avg_shortest_path_overall)


# 12. Density
density_attack = nx.density(AttackG_5_5)
density_non_attack = nx.density(nonAttackG_5_5)
density_overall = nx.density(overallG_5_5)

print("Density for attack graph:", density_attack)
print("Density for non-attack graph:", density_non_attack)
print("Density for overall graph:", density_overall)

# 13. Diameter
diameter_attack = calculate_diameter(AttackG_5_5)
diameter_non_attack = calculate_diameter(nonAttackG_5_5)
diameter_overall = calculate_diameter(overallG_5_5)

print("Diameter for attack graph:", diameter_attack)
print("Diameter for non-attack graph:", diameter_non_attack)
print("Diameter for overall graph:", diameter_overall)


# 14. Number of Louvain communities detected
num_louvain_attack = detect_communities(AttackG_5_5)
num_louvain_non_attack = detect_communities(nonAttackG_5_5)
num_louvain_overall = detect_communities(overallG_5_5)

print("Number of Louvain communities detected in attack graph:", num_louvain_attack)
print("Number of Louvain communities detected in non-attack graph:", num_louvain_non_attack)
print("Number of Louvain communities detected in overall graph:", num_louvain_overall)

# 15. Number of strongly connected components
num_strongly_connected_attack = nx.number_strongly_connected_components(AttackG_5_5)
num_strongly_connected_non_attack = nx.number_strongly_connected_components(nonAttackG_5_5)
num_strongly_connected_overall = nx.number_strongly_connected_components(overallG_5_5)

print("Number of strongly connected components in attack graph:", num_strongly_connected_attack)
print("Number of strongly connected components in non-attack graph:", num_strongly_connected_non_attack)
print("Number of strongly connected components in overall graph:", num_strongly_connected_overall)

# 16. Number of weakly connected components
num_weakly_connected_attack = nx.number_weakly_connected_components(AttackG_5_5)
num_weakly_connected_non_attack = nx.number_weakly_connected_components(nonAttackG_5_5)
num_weakly_connected_overall = nx.number_weakly_connected_components(overallG_5_5)

print("Number of weakly connected components in attack graph:", num_weakly_connected_attack)
print("Number of weakly connected components in non-attack graph:", num_weakly_connected_non_attack)
print("Number of weakly connected components in overall graph:", num_weakly_connected_overall)




reciprocity_attack = calculate_reciprocity(AttackG_5_5)
reciprocity_non_attack = calculate_reciprocity(nonAttackG_5_5)
reciprocity_overall = calculate_reciprocity(overallG_5_5)

print("Reciprocity for attack graph:", reciprocity_attack)
print("Reciprocity for non-attack graph:", reciprocity_non_attack)
print("Reciprocity for overall graph:", reciprocity_overall)

c:\ProgramData\anaconda3\Lib\site-packages\networkx\algorithms\assortativity\mixing.py:207: RuntimeWarning: invalid value encountered in divide
  a = a / a.sum()


Assortativity coefficient for attack graph: -0.09740248973191355
Assortativity coefficient for non-attack graph: nan
Assortativity coefficient for overall graph: 0.08550032842798314
Average betweenness centrality for attack graph: 0.0
Average betweenness centrality for non-attack graph: 0.0
Average betweenness centrality for overall graph: 2.4612429754605663e-10
Average closeness centrality for attack graph: 0.014050613258810758
Average closeness centrality for non-attack graph: 0.0
Average closeness centrality for overall graph: 0.0005431591599152191
Average clustering coefficient for attack graph: 0.0
Average clustering coefficient for non-attack graph: 0.0
Average clustering coefficient for overall graph: 0.0
Average degree for attack graph: 3.175438596491228
Average degree for non-attack graph: 0.0
Average degree for overall graph: 2.750591949486977
Average degree centrality for attack graph: 0.028101226517621425
Average degree centrality for non-attack graph: 0.0
Average degree ce